# Data-context

In [ ]:
import torch
from torch.utils.data import Dataset
import pickle
import numpy as np
import json
import random


seed = 42
random.seed(seed)

In [ ]:
def cal_dataset_stats(dataset, keys):
	
	v = []
	a = []
	for key in keys:
		if key not in dataset.keys():
			continue
		if dataset[key]["v"] is not None:
			for visual in dataset[key]["v"]:
				if visual.shape[0] > 0:
					v.append(visual)
		
		for acoustic in dataset[key]["a"]:
			if acoustic.shape[0] > 0:
				a.append(acoustic)

	mean_v = np.mean(np.concatenate(v, 0), 0)
	std_v = np.std(np.concatenate(v, 0), 0)


	mean_a = np.mean(np.concatenate(a, 0), 0)
	std_a = np.std(np.concatenate(a, 0), 0)


	return (0, 1, mean_a, std_a)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Majorproject/intention/data_resource")

In [ ]:
with open("SentiWords_1.1.txt", "r") as f:
	lines = f.readlines()

In [ ]:
positive_words = []
for line in lines:
	if not line.startswith("#"):
		word, score = line.strip().split("\t")
		if float(score) > 0:
		 	positive_words.append([word.split("#")[0].lower(), float(score)])

In [ ]:
with open("dataset_indices.pickle", "rb") as f:
	split_indices = pickle.load(f) # splitting the data 5 times with 552 dialouges for training and 138 for testing

In [ ]:
with open("vat_ferplus.dataset", "rb") as f:
	dataset = pickle.load(f)                    # resnet, ferplus

In [ ]:
with open("last_bert.dataset", "rb") as f:
	bert_words = pickle.load(f)

In [ ]:
# global features
with open("bert_global_embedding.pickle", "rb") as f:
	bert_global_embedding = pickle.load(f)


with open("resnet_utterance.pickle", "rb") as f:
	resnet_global_embedding = pickle.load(f)


with open("opensmile_utterance.dataset", "rb") as f:
	opensmile_global_embedding = pickle.load(f)

with open("audio_mask.pickle", "rb") as f:
	_audio_mask = pickle.load(f)

audio_masks = []

for i in range(1, 6):
	audio_masks.append(_audio_mask[i])

audio_masks.append(_audio_mask[0])

In [ ]:
# context information
with open("context_bert_all.dataset", "rb") as f:
	bert_context_embedding = pickle.load(f)

with open("context_resnet.pickle", "rb") as f:
	resnet_context_embedding = pickle.load(f)

with open("context_audio.pickle", "rb") as f:
	old_context_audio = pickle.load(f)

audio_context_embedding = {}

for key in old_context_audio.keys():
	audio_context_embedding[key[:-2]] = old_context_audio[key]


with open("context_audio_mask.pickle", "rb") as f:
	_audio_context_mask = pickle.load(f)

audio_context_mask = []

for i in range(1, 6):
	audio_context_mask.append(_audio_context_mask[i])

audio_context_mask.append(_audio_context_mask[0])

In [ ]:
keys = list(dataset.keys())

In [ ]:
with open("text_data.json") as f:
	json_data = json.load(f)

In [ ]:
train_keys = []
test_keys = []


for key in dataset.keys():
	if json_data[key]["show"] == "FRIENDS":
		test_keys.append(key)
	else:
		train_keys.append(key)
split_indices.append([train_keys, test_keys])

In [ ]:
# For speaker dependant
speakers = ["PERSON"]
for key in json_data.keys():
	sp = json_data[key]["speaker"]
	if "PERSON" not in sp:
		speakers.append(sp)

sps = list(set(speakers))

In [ ]:
speakers_reps = {}

for idx, sp in enumerate(sps):
	spr = np.zeros((1, len(sps)))
	spr[0][idx] = 1
	speakers_reps[sp] = spr

In [ ]:
speakers_dic = []

dataset_splits = []

dataset_stats = []

# splitting the dataset for train, test and validation
for i in range(6):
  train_keys, test_keys = split_indices[i]
  random.shuffle(train_keys)

  lens = int(len(train_keys) * 0.8)

  train_set = []
  for key in train_keys[:lens]:
    if key in keys:
      train_set.append(key)

  dev_set = []
  for key in train_keys[lens:]:
    if key in keys:
      dev_set.append(key)

  test_set = []
  for key in test_keys:
    if key in keys:
      test_set.append(key)

  all_train_set = []
  for key in train_keys:
    if key in keys:
      all_train_set.append(key)


  isp = []
  for key in train_set:
    isp.append(json_data[key]["speaker"])

  all_isp = []
  for key in all_train_set:
    all_isp.append(json_data[key]["speaker"])



  dataset_splits.append({"train":train_set, "dev":dev_set, "test":test_set, "all_train":all_train_set})

  dataset_stats.append({"train":cal_dataset_stats(dataset, train_set), "all_train":cal_dataset_stats(dataset, all_train_set)})

  speakers_dic.append({"train":list(set(isp)), "all_train":list(set(isp))})

In [ ]:
context_dataset_stats = []

for i in range(6):
	train_keys, test_keys = split_indices[i]
	all_train_set = []
	for key in train_keys:
		if key in keys:
			all_train_set.append(key)


	v = []
	a = []
	t = []
	for key in all_train_set:
		v.append(resnet_context_embedding[key])
		a.append(audio_context_embedding[key][:,audio_context_mask[i]])
		t.append(bert_context_embedding[key])

	mean_v = np.mean(np.concatenate(v, 0), 0)
	std_v = np.std(np.concatenate(v, 0), 0)

	mean_a = np.mean(np.concatenate(a, 0), 0)
	std_a = np.std(np.concatenate(a, 0), 0)

	mean_t = np.mean(np.concatenate(t, 0), 0)
	std_t = np.std(np.concatenate(t, 0), 0)

	context_dataset_stats.append([0, 1, 0, 1, 0, 1])

In [ ]:
class IntentionDataset(Dataset):
	def __init__(self, dname="train", idx=0, stat_name="train", p_value=0.5):
		self.dataset = dataset
		self.dname = dname
		self.vdims = 35
		self.adims = 65
		self.tdims = 768
		self.dia_names = dataset_splits[idx][self.dname]
		self.state = dataset_stats[idx][stat_name]
		self.audio_mask = audio_masks[idx]
		self.audio_mask_context = audio_context_mask[idx]
		self.positive_words = [w for w,s in positive_words if s > p_value]
		self.context_stat = context_dataset_stats[idx]
		self.speaker_dict = speakers_dic[idx][stat_name]

	def __getitem__(self, idx):

		utt_name = self.dia_names[idx]        # Utterance id at a particular index

		speaker_name = json_data[utt_name]["speaker"]      # speaker name for that utterance

		if speaker_name in self.speaker_dict:
			if "PERSON" in speaker_name:
				speaker_name = "PERSON"
			speaker_vector = speakers_reps[speaker_name]
		else:
			speaker_vector = speakers_reps["PERSON"]

		speaker_vector = torch.from_numpy(speaker_vector).float()    # Creating a tensor from numpy array
		# print(utt_name)

		feat = self.dataset[utt_name]        # features for that utterance

		visual_feats = []

		if 	feat["v"] is not None:
			for feats in feat["v"]:
				if feats.shape[0] > 0:
					visual_feats.append((feats-self.state[0])/self.state[1])         # Standardization
				else:
					visual_feats.append(feats)


		acoustic_feats = []

		if 	feat["a"] is not None:
			for feats in feat["a"]:
				if feats.shape[0] > 0:
					acoustic_feats.append((feats-self.state[2])/self.state[3])        # Standardization
				else:
					acoustic_feats.append(feats)

		textual_feats = bert_words[utt_name]["t"]#feat["t"]

		# print(len(bert_words[utt_name]), len(visual_feats))
		# if len(bert_words[utt_name]) != len(visual_feats):
		# 	print(len(bert_words[utt_name]), len(len(visual_feats)))
		# 	print("error")
		# 	exit()
			

		bert_context = torch.from_numpy((bert_context_embedding[utt_name]-self.context_stat[4])/self.context_stat[5]).float()    # Standardization

		bert_global = torch.from_numpy(bert_global_embedding[utt_name]).float()

		# 1000
		audio_global = torch.from_numpy(opensmile_global_embedding[utt_name][:,self.audio_mask]).float()
		audio_context = torch.from_numpy((audio_context_embedding[utt_name][:,self.audio_mask_context]-self.context_stat[2])/self.context_stat[3]).float()


		# 2048
		if utt_name in resnet_global_embedding.keys():
			video_global = torch.from_numpy(resnet_global_embedding[utt_name]).view(1, -1).float()
		else:
			video_global = torch.zeros((1, 2048))


		video_context = torch.from_numpy((resnet_context_embedding[utt_name]-self.context_stat[0])/self.context_stat[1]).view(1, -1).float()



		mask = [int(w in self.positive_words) for w in feat["words"]]     # mask for each utterance in the dataset

		label = int(json_data[utt_name]["intention"])                     # label for that utterance True/False

		example = {"visual_feats":visual_feats, "acoustic_feats":acoustic_feats, "textual_feats":textual_feats, 
				"mask":mask, "bert_global":bert_global, "audio_global":audio_global, "video_global":video_global,"label":label, "bert_context":bert_context,  "audio_context":audio_context,
				"video_context":video_context, "speaker_vector":speaker_vector}

		return example


	def __len__(self):
		return len(self.dia_names)

In [ ]:
def collate_fn(batch):

	vdims = 2048
	adims = 65
	tdims = 768


	visual_nums = []
	acoustic_nums = []

	text_nums = []


	visual_flags = []
	acoustic_flags = []

	bert_global_reps = []
	audio_global_reps = []	
	video_global_reps = []


	bert_context_reps = []
	audio_context_reps = []	
	video_context_reps = []

	speaker_vectors = []

	for example in batch:

		bert_global_reps.append(example["bert_global"].view(1,-1))
		audio_global_reps.append(example["audio_global"])
		video_global_reps.append(example["video_global"])

		bert_context_reps.append(example["bert_context"])
		audio_context_reps.append(example["audio_context"])
		video_context_reps.append(example["video_context"])

		speaker_vectors.append(example["speaker_vector"])		


		acoustic_nums.append(max([len(v) for v in example["acoustic_feats"]]))

		acoustic_flag = []
		for i in range(len(example["acoustic_feats"])):
			acoustic_flag.append(1)


		text_nums.append(len(example["textual_feats"]))

		visual_feats = example["visual_feats"]
		visual_flag = []

		if len(visual_feats) == 0:
			visual_nums.append(1)
			temp = []
			for i in range(len(example["textual_feats"])):
				temp.append(np.empty((0,vdims)))
				visual_flag.append(0)
			example["visual_feats"] = temp

		
		else:
			visual_nums.append(max([len(v) for v in example["visual_feats"]]))
			for i in range(len(example["visual_feats"])):
				visual_flag.append(1)


		visual_flags.append(visual_flag)
		acoustic_flags.append(acoustic_flag)

	bert_global_reps = torch.cat(bert_global_reps, 0)
	audio_global_reps = torch.cat(audio_global_reps, 0)
	video_global_reps = torch.cat(video_global_reps, 0)

	speaker_vectors = torch.cat(speaker_vectors, 0)

	
	bert_context_reps = torch.cat(bert_context_reps, 0)
	audio_context_reps = torch.cat(audio_context_reps, 0)
	video_context_reps = torch.cat(video_context_reps, 0)


	if max(acoustic_nums) == 0:
		acoustic_nums.append(1)

	if max(visual_nums) == 0:
		visual_nums.append(1)


	bsz = len(batch)
	MAX_WORD = min(128, max(text_nums))

	# visual feats speakers
	MAX_LEN = min(100, max(visual_nums))
	video_tensor = torch.zeros((bsz, MAX_WORD, MAX_LEN, vdims))

	for i_batch in range(bsz):
		for i_utt, input_row in enumerate(batch[i_batch]["visual_feats"]):
			video_tensor[i_batch, i_utt, :min(MAX_LEN, len(input_row))] = torch.Tensor(input_row[:min(MAX_LEN, len(input_row)), -vdims:])

	vlens = []
	for i_batch in range(bsz):
		vlen = []
		for input_row in batch[i_batch]["visual_feats"]:
			vlen.append(max(1, min(MAX_LEN, len(input_row))))

		vlen = vlen[:MAX_WORD]
		now_len = len(vlen)
		for ii in range(MAX_WORD-now_len):
			vlen.append(1)

		vlens.append(vlen)

	# acoustic features speakers 
	MAX_LEN = min(300, max(acoustic_nums))
	audio_tensor = torch.zeros((bsz, MAX_WORD, MAX_LEN, adims))

	for i_batch in range(bsz):
		for i_utt, input_row in enumerate(batch[i_batch]["acoustic_feats"]):
			audio_tensor[i_batch, i_utt, :min(MAX_LEN, len(input_row))] = torch.Tensor(input_row[:min(MAX_LEN, len(input_row))])


	alens = []
	for i_batch in range(bsz):
		alen = []
		for input_row in batch[i_batch]["acoustic_feats"]:
			alen.append(max(1, min(MAX_LEN, len(input_row))))

		alen = alen[:MAX_WORD]
		now_len = len(alen)
		for ii in range(MAX_WORD-now_len):
			alen.append(1)

		alens.append(alen)


	# textual features speakers 
	MAX_LEN = MAX_WORD
	text_tensor = torch.zeros((bsz, MAX_WORD, tdims))

	for i_batch in range(bsz):
		for i_utt, input_row in enumerate(batch[i_batch]["textual_feats"]):
			text_tensor[i_batch, i_utt, :min(MAX_LEN, len(input_row))] = torch.Tensor(input_row[:min(MAX_LEN, len(input_row))])

	tlens = []
	for i_batch in range(bsz):
		tlens.append(min(MAX_WORD, len(batch[i_batch]["textual_feats"])))



	masks = []
	for i_batch in range(bsz):

		mask = []
		for m in batch[i_batch]["mask"]:
			mask.append(m)

		mask = mask[:MAX_WORD]
		now_len = len(mask)
		for ii in range(MAX_WORD-now_len):
			mask.append(0)

		masks.append(mask)

	v_masks = []
	for i_batch in range(bsz):
		mask = []
		for m in visual_flags[i_batch]:
			mask.append(m)

		mask = mask[:MAX_WORD]
		now_len = len(mask)
		for ii in range(MAX_WORD-now_len):
			mask.append(0)
		v_masks.append(mask)


	a_masks = []
	for i_batch in range(bsz):
		mask = []
		for m in acoustic_flags[i_batch]:
			mask.append(m)

		mask = mask[:MAX_WORD]
		now_len = len(mask)
		for ii in range(MAX_WORD-now_len):
			mask.append(0)
		a_masks.append(mask)


	labels = []
	for i_batch in range(bsz):
		labels.append(batch[i_batch]["label"])


	labels = torch.Tensor(labels).long()
	masks = torch.Tensor(masks).float()
	v_masks = torch.Tensor(masks).float()
	a_masks = torch.Tensor(masks).float()

	return  speaker_vectors, bert_context_reps,  audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, labels


# Model

In [ ]:
import numpy as np
seed = 42
np.random.seed(seed)
import random
import torch
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import utils as nn_utils

In [ ]:
class TAttention_Mask_Relu_Score(nn.Module):
	def __init__(self):
		super(TAttention_Mask_Relu_Score, self).__init__()
		config = {"input_dims":[768, 65, 2048], "global_input_dims":[768, 1000, 2048],"hidden_dims": 200, "output_dims": 2, "dpout":0.5}
		dt, da, dv = config["input_dims"]
		gdt, gda, gdv = config["global_input_dims"]
		h = config["hidden_dims"]
		output_dim = config["output_dims"]
		dropout = config["dpout"]

		self.linear_t = nn.Linear(dt, h)
		self.linear_v = nn.Linear(dv, h)
		self.linear_a = nn.Linear(da, h)

		self.importance_ta = nn.Linear(h+h, 1)
		self.importance_tv = nn.Linear(h+h, 1)
		self.importance_t = nn.Linear(h, 1)

		self.fc = nn.Linear(gdt+gda+gdv+dt+da+dv+gdt+gdv, output_dim)
		self.dropout = nn.Dropout(dropout)
		self.h = h


	def forward(self, speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, get_feature=False):
		
		batch_size, word_num,  _ = text_tensor.size()


		vl = video_tensor.size(2)
		al = audio_tensor.size(2)

		global_h1 = torch.zeros(1, batch_size, self.h)
		if torch.cuda.is_available():
			global_h1 = global_h1.cuda()


		vvlens = []
		for vlen in vlens:
			for l in vlen:
				vvlens.append(l)

		vvlens_tensor = torch.Tensor(vvlens)
		if torch.cuda.is_available():
			vvlens_tensor = vvlens_tensor.cuda()

		video_tensor = video_tensor.contiguous().view(batch_size*word_num, vl, -1)
		video_tensor = torch.sum(video_tensor, 1)/vvlens_tensor.view(batch_size*word_num, 1).expand(batch_size*word_num, video_tensor.size(-1))
		v_last_hs = F.tanh(self.linear_v(video_tensor))
		v_last_hs = v_last_hs.contiguous().view(batch_size, word_num, -1)


		aalens = []
		for alen in alens:
			for l in alen:
				aalens.append(l)


		aalens_tensor = torch.Tensor(aalens)
		if torch.cuda.is_available():
			aalens_tensor = aalens_tensor.cuda()

		audio_tensor = audio_tensor.contiguous().view(batch_size*word_num, al, -1)
		audio_tensor = torch.sum(audio_tensor, 1)/aalens_tensor.view(batch_size*word_num, 1).expand(batch_size*word_num, audio_tensor.size(-1))
		a_last_hs = F.tanh(self.linear_a(audio_tensor))
		a_last_hs = a_last_hs.contiguous().view(batch_size, word_num, -1)


		out_t = F.tanh(self.linear_t(text_tensor))

		output_tensor = torch.cat([text_tensor, audio_tensor.view(batch_size, word_num, -1), video_tensor.view(batch_size, word_num, -1)], -1)


		importance_tv = F.relu(self.importance_tv(torch.cat([out_t, v_last_hs], -1)))
		importance_ta = F.relu(self.importance_ta(torch.cat([out_t, a_last_hs], -1)))

		# masking
		index = torch.arange(word_num).view(1,-1).expand(batch_size, word_num)
		tlens_tensor = torch.Tensor(tlens).unsqueeze(1).expand(batch_size, word_num)
		if torch.cuda.is_available():
			tlens_tensor = tlens_tensor.cuda()
			index = index.cuda()

		importance_tv = importance_tv.squeeze(2) * v_masks * masks

		importance_ta = importance_ta.squeeze(2) * a_masks * masks

		importance_t = self.importance_t(out_t)
		importance_t = importance_t.squeeze(2)

		importances =  importance_t + importance_tv + importance_ta + (1-(index < tlens_tensor).float()) * (-1e8)

		global_weights = F.softmax(importances, 1)

		last_hs = torch.sum(global_weights.unsqueeze(2).expand_as(output_tensor) * output_tensor, 1)

		global_inputs = torch.cat([bert_global_reps, audio_global_reps, video_global_reps, bert_context_reps, video_context_reps], -1)

		last_hs = torch.cat([last_hs, global_inputs], -1)


		last_hs = F.tanh(last_hs)

		if get_feature:
			return last_hs

		last_hs = self.dropout(last_hs)
		output = self.fc(last_hs)
		return output

In [ ]:
def get_model():
	return TAttention_Mask_Relu_Score()

# Training

In [ ]:
import numpy as np
seed = 123
np.random.seed(seed)
import random
import torch
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,\
                        classification_report, precision_recall_fscore_support
import tqdm
import time

from torch.utils.data import DataLoader

In [ ]:
def eval(model, loss_fn, dataset):
	model.eval()
	gds = []
	preds = []
	losses = []
	for speaker_vectors, bert_context_reps,  audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, labels in tqdm.tqdm(dataset):
		if torch.cuda.is_available():
			speaker_vectors = speaker_vectors.cuda()
			bert_context_reps = bert_context_reps.cuda()
			audio_context_reps = audio_context_reps.cuda()
			video_context_reps = video_context_reps.cuda()
			bert_global_reps = bert_global_reps.cuda()
			audio_global_reps = audio_global_reps.cuda()
			video_global_reps = video_global_reps.cuda()					
			video_tensor = video_tensor.cuda()
			audio_tensor = audio_tensor.cuda()
			text_tensor = text_tensor.cuda()
			labels = labels.cuda()
			masks = masks.cuda()
			v_masks = v_masks.cuda()
			a_masks = a_masks.cuda()


		output = model(speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks)

		loss = loss_fn(output, labels)
		losses.append(loss.detach().cpu().numpy())		

		gds.append(labels.cpu().numpy())
		pred = torch.argmax(output,1)
		preds.append(pred.cpu().numpy())



	preds  = np.concatenate(preds)
	gds = np.concatenate(gds)

	avg_loss = round(np.average(losses),4)
	avg_accuracy = round(accuracy_score(gds,preds)*100,2)
	avg_fscore = round(f1_score(gds,preds,average='weighted')*100,2)
	class_report = classification_report(gds,preds,digits=4)

	return avg_loss, avg_accuracy, avg_fscore, class_report

In [ ]:
results = []
for dataset_index in tqdm.trange(6):
		intention_train = DataLoader(IntentionDataset("train", dataset_index, "train", 0.5), batch_size=32, shuffle=True, collate_fn=collate_fn)
		intention_dev = DataLoader(IntentionDataset("dev", dataset_index, "train", 0.5), batch_size=32, shuffle=True, collate_fn=collate_fn)
		intention_test = DataLoader(IntentionDataset("test", dataset_index, "all_train", 0.5), batch_size=32, shuffle=True, collate_fn=collate_fn)
		print("Argument file decoded" , 0.5)
		print("Arguments bz" , 32)
		print("Arguments epochs" , 100)
		# print("Arguments model" , "")
		print("Arguments lr" , 0.0001)
		# best_score = 0
		results_epoch = []
		epochs = 100

		model = get_model()
		print("Model file" , model)
		if torch.cuda.is_available():
			model = model.cuda()

		loss_fn = nn.CrossEntropyLoss()
		token_loss_fn = nn.CrossEntropyLoss(ignore_index=5)


		optimizer = optim.Adam(model.parameters(), lr=0.0001)

		for epoch in tqdm.trange(epochs):
			model.train()

			for speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, labels in tqdm.tqdm(intention_train):
				if torch.cuda.is_available():
					speaker_vectors = speaker_vectors.cuda()
					bert_context_reps = bert_context_reps.cuda()
					audio_context_reps = audio_context_reps.cuda()
					video_context_reps = video_context_reps.cuda()
					audio_global_reps = audio_global_reps.cuda()
					video_global_reps = video_global_reps.cuda()
					bert_global_reps = bert_global_reps.cuda()
					video_tensor = video_tensor.cuda()
					audio_tensor = audio_tensor.cuda()
					text_tensor = text_tensor.cuda()
					masks = masks.cuda()
					v_masks = v_masks.cuda()
					a_masks = a_masks.cuda()
					labels = labels.cuda()

				# print("speaker vectors himanshu" , speaker_vectors)

				output = model(speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks)
				loss = loss_fn(output, labels)


				loss.backward()	
				optimizer.step()
				optimizer.zero_grad()


			avg_loss, avg_accuracy, avg_fscore, class_report = eval(model, loss_fn, intention_dev)
			print(" [DEV] \n")
			print(" loss : {}, accuracy : {}, f1-score :{} \n".format(avg_loss, avg_accuracy, avg_fscore))
			print(class_report)

			results_epoch.append(avg_fscore)


		
		final_epochs = np.argmax(np.array(results_epoch), 0)
		final_epochs += 1

		print("Epochs Max nums is {}\n".format(final_epochs))
		print("Start using All data \n")


		intention_train = DataLoader(IntentionDataset("all_train", dataset_index, "all_train", 0.5), batch_size=32, shuffle=True, collate_fn=collate_fn)

		final_model = get_model()

		if torch.cuda.is_available():
			final_model = final_model.cuda()

		loss_fn = nn.CrossEntropyLoss()
		token_loss_fn = nn.CrossEntropyLoss(ignore_index=5)

		optimizer = optim.Adam(final_model.parameters(), lr=0.0001)

		for epoch in tqdm.trange(final_epochs):
			final_model.train()

			for speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, labels in tqdm.tqdm(intention_train):
				if torch.cuda.is_available():
					speaker_vectors = speaker_vectors.cuda()
					bert_context_reps = bert_context_reps.cuda()
					audio_context_reps = audio_context_reps.cuda()
					video_context_reps = video_context_reps.cuda()
					bert_global_reps = bert_global_reps.cuda()
					audio_global_reps = audio_global_reps.cuda()
					video_global_reps = video_global_reps.cuda()
					video_tensor = video_tensor.cuda()
					audio_tensor = audio_tensor.cuda()
					text_tensor = text_tensor.cuda()
					labels = labels.cuda()
					masks = masks.cuda()
					v_masks = v_masks.cuda()
					a_masks = a_masks.cuda()



				output = final_model(speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks)
				loss = loss_fn(output, labels)

				loss.backward()	
				optimizer.step()
				optimizer.zero_grad()

		torch.save(final_model.state_dict(), '{}_{}.pkl'.format("result_model", dataset_index))

		avg_loss, avg_accuracy, avg_fscore, class_report = eval(final_model, loss_fn, intention_test)

		print(" [TEST-{}] \n".format(dataset_index))
		print(" loss : {}, accuracy : {}, f1-score :{} \n".format(avg_loss, avg_accuracy, avg_fscore))
		print(class_report)


		results.append([avg_loss, avg_accuracy, avg_fscore])

                  
result = np.mean(np.array(results[:5]), 0)


print("\n")
print( "[Final Result] \n")
print(results[-1])
print(" loss : {}, accuracy : {}, f1-score :{} \n".format(results[-1][0], results[-1][1], results[-1][2]))
print("\n")

  0%|          | 0/6 [00:00<?, ?it/s]

Argument file decoded 0.5
Arguments bz 32
Arguments epochs 100
Arguments lr 0.0001
Model file TAttention_Mask_Relu_Score(
  (linear_t): Linear(in_features=768, out_features=200, bias=True)
  (linear_v): Linear(in_features=2048, out_features=200, bias=True)
  (linear_a): Linear(in_features=65, out_features=200, bias=True)
  (importance_ta): Linear(in_features=400, out_features=1, bias=True)
  (importance_tv): Linear(in_features=400, out_features=1, bias=True)
  (importance_t): Linear(in_features=200, out_features=1, bias=True)
  (fc): Linear(in_features=9513, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)



  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


  7%|▋         | 1/14 [00:03<00:42,  3.26s/it]

 14%|█▍        | 2/14 [00:04<00:21,  1.78s/it]

 21%|██▏       | 3/14 [00:04<00:11,  1.07s/it]

 29%|██▊       | 4/14 [00:04<00:07,  1.33it/s]

 36%|███▌      | 5/14 [00:04<00:05,  1.79it/s]

 43%|████▎     | 6/14 [00:05<00:04,  1.98it/s]

 50%|█████     | 7/14 [00:05<00:02,  2.48it/s]

 57%|█████▋    | 8/14 [00:05<00:02,  2.77it/s]

 64%|██████▍   | 9/14 [00:05<00:01,  2.82it/s]

 71%|███████▏  | 10/14 [00:06<00:01,  3.30it/s]

 79%|███████▊  | 11/14 [00:06<00:00,  3.64it/s]

 86%|████████▌ | 12/14 [00:06<00:00,  3.99it/s]

 93%|█████████▎| 13/14 [00:06<00:00,  4.40it/s]

100%|██████████| 14/14 [00:06<00:00,  2.05it/s]


  0%|          | 0/4 [00

 [DEV] 

 loss : 0.7245000004768372, accuracy : 44.14, f1-score :27.04 

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        62
           1     0.4414    1.0000    0.6125        49

    accuracy                         0.4414       111
   macro avg     0.2207    0.5000    0.3063       111
weighted avg     0.1949    0.4414    0.2704       111





  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


  7%|▋         | 1/14 [00:00<00:06,  1.94it/s]

 14%|█▍        | 2/14 [00:00<00:04,  2.64it/s]

 21%|██▏       | 3/14 [00:00<00:02,  3.75it/s]

 29%|██▊       | 4/14 [00:01<00:02,  4.55it/s]

 36%|███▌      | 5/14 [00:01<00:01,  4.77it/s]

 43%|████▎     | 6/14 [00:02<00:03,  2.57it/s]

 50%|█████     | 7/14 [00:02<00:02,  2.87it/s]

 57%|█████▋    | 8/14 [00:02<00:01,  3.19it/s]

 64%|██████▍   | 9/14 [00:02<00:01,  3.74it/s]

 71%|███████▏  | 10/14 [00:02<00:00,  4.15it/s]

 79%|███████▊  | 11/14 [00:03<00:00,  3.77it/s]

 86%|████████▌ | 12/14 [00:03<00:00,  4.26it/s]

 93%|█████████▎| 13/14 [00:03<00:00,  4.35it/s]

100%|██████████| 14/14 [00:03<00:00,  3.77it/s]


  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:

...

 [DEV] 

 loss : 0.6067000031471252, accuracy : 59.7, f1-score :59.57 

              precision    recall  f1-score   support

           0     0.5000    0.4815    0.4906        27
           1     0.6585    0.6750    0.6667        40

    accuracy                         0.5970        67
   macro avg     0.5793    0.5782    0.5786        67
weighted avg     0.5946    0.5970    0.5957        67





  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 11%|█         | 1/9 [00:00<00:01,  4.43it/s]

 22%|██▏       | 2/9 [00:00<00:01,  4.20it/s]

 33%|███▎      | 3/9 [00:00<00:01,  3.58it/s]

 44%|████▍     | 4/9 [00:00<00:01,  4.10it/s]

 56%|█████▌    | 5/9 [00:01<00:01,  3.86it/s]

 67%|██████▋   | 6/9 [00:01<00:01,  2.66it/s]

 78%|███████▊  | 7/9 [00:02<00:00,  2.11it/s]

 89%|████████▉ | 8/9 [00:02<00:00,  2.50it/s]

100%|██████████| 9/9 [00:02<00:00,  3.09it/s]


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  3.58it/s]

100%|██████████| 3/3 [00:01<00:00,  2.48it/s]

100%|██████████| 100/100 [06:47<00:00,  4.07s/it]


 [DEV] 

 loss : 0.7490000128746033, accuracy : 59.7, f1-score :59.57 

              precision    recall  f1-score   support

           0     0.5000    0.4815    0.4906        27
           1     0.6585    0.6750    0.6667        40

    accuracy                         0.5970        67
   macro avg     0.5793    0.5782    0.5786        67
weighted avg     0.5946    0.5970    0.5957        67

Epochs Max nums is 34

Start using All data 




  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


  9%|▉         | 1/11 [00:00<00:03,  3.03it/s]

 18%|█▊        | 2/11 [00:00<00:03,  2.40it/s]

 27%|██▋       | 3/11 [00:01<00:03,  2.15it/s]

 36%|███▋      | 4/11 [00:01<00:03,  2.22it/s]

 45%|████▌     | 5/11 [00:02<00:03,  1.77it/s]

 55%|█████▍    | 6/11 [00:02<00:02,  2.21it/s]

 64%|██████▎   | 7/11 [00:03<00:01,  2.38it/s]

 73%|███████▎  | 8/11 [00:03<00:01,  2.61it/s]

 82%|████████▏ | 9/11 [00:03<00:00,  3.05it/s]

100%|██████████| 11/11 [00:04<00:00,  2.72it/s]

  3%|▎         | 1/34 [00:04<02:13,  4.06s/it]

  0%|          | 0/11 [00:00<?, ?it/s]

  9%|▉         | 1/11 [00:00<00:02,  3.43it/s]

 18%|█▊        | 2/11 [00:00<00:02,  3.84it/s]

 27%|██▋       | 3/11 [00:00<00:02,  3

 [TEST-5] 

 loss : 0.6212999820709229, accuracy : 67.04, f1-score :66.76 

              precision    recall  f1-score   support

           0     0.8162    0.5468    0.6549       203
           1     0.5799    0.8355    0.6846       152

    accuracy                         0.6704       355
   macro avg     0.6980    0.6912    0.6698       355
weighted avg     0.7150    0.6704    0.6676       355



[Final Result] 

[0.6213, 67.04, 66.76]
 loss : 0.6212999820709229, accuracy : 67.04, f1-score :66.76 





# Combining the results for classification

In [ ]:
import numpy as np
seed = 123
np.random.seed(seed)
import random
import torch
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,\
                        classification_report, precision_recall_fscore_support
import tqdm
import time

from torch.utils.data import DataLoader

import pickle

In [ ]:
train_dataset = []
test_dataset = []


for dataset_index in tqdm.trange(6):
		intention_test = DataLoader(IntentionDataset("test", dataset_index, "all_train", 0.5), batch_size=32, shuffle=True, collate_fn=collate_fn)
		intention_train = DataLoader(IntentionDataset("all_train", dataset_index, "all_train", 0.5), batch_size=32, shuffle=True, collate_fn=collate_fn)

		final_model = get_model()

		# if torch.cuda.is_available():
		# 	final_model = final_model.cuda()

		final_model.load_state_dict(torch.load('{}_{}.pkl'.format("result_model", dataset_index), map_location=torch.device('cpu')))


		final_model.eval()

		features = []
		true_labels = []

		for speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, labels in tqdm.tqdm(intention_train):
			# if torch.cuda.is_available():
			speaker_vectors = speaker_vectors
			bert_context_reps = bert_context_reps
			audio_context_reps = audio_context_reps
			video_context_reps = video_context_reps

			bert_global_reps = bert_global_reps
			audio_global_reps = audio_global_reps
			video_global_reps = video_global_reps
			video_tensor = video_tensor
			audio_tensor = audio_tensor
			text_tensor = text_tensor
			labels = labels
			masks = masks
			v_masks = v_masks
			a_masks = a_masks


			output = final_model(speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, get_feature=True)

			output = output.detach().cpu()				
			features.append(output.numpy())
			true_labels.append(labels.detach().cpu().numpy())

		train_dataset.append([np.concatenate(features), np.concatenate(true_labels)])



		features = []
		true_labels = []

		for speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, labels in tqdm.tqdm(intention_test):
			# if torch.cuda.is_available():
			speaker_vectors = speaker_vectors
			bert_context_reps = bert_context_reps
			audio_context_reps = audio_context_reps
			video_context_reps = video_context_reps
			bert_global_reps = bert_global_reps
			audio_global_reps = audio_global_reps
			video_global_reps = video_global_reps
			video_tensor = video_tensor
			audio_tensor = audio_tensor
			text_tensor = text_tensor
			labels = labels
			masks = masks
			v_masks = v_masks
			a_masks = a_masks


			output = final_model(speaker_vectors, bert_context_reps, audio_context_reps, video_context_reps, bert_global_reps, audio_global_reps, video_global_reps, video_tensor, vlens, audio_tensor, alens, text_tensor, tlens, masks, v_masks, a_masks, get_feature=True)
			

			output = output.detach().cpu()
			features.append(output.numpy())
			true_labels.append(labels.detach().cpu().numpy())

		test_dataset.append([np.concatenate(features), np.concatenate(true_labels)])


with open("{}.pickle".format("model_result"), "wb") as f:
		pickle.dump([train_dataset, test_dataset], f)

  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")

100%|██████████| 18/18 [00:04<00:00,  4.09it/s]

100%|██████████| 18/18 [00:03<00:00,  4.67it/s]

100%|██████████| 18/18 [00:03<00:00,  4.87it/s]

100%|██████████| 18/18 [00:04<00:00,  4.39it/s]

100%|██████████| 18/18 [00:04<00:00,  4.17it/s]

100%|██████████| 11/11 [00:03<00:00,  3.29it/s]

100%|██████████| 6/6 [00:33<00:00,  5.63s/it]
